In [1]:
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
print(tf.__version__)
sess = tf.Session()

1.13.1


In [3]:
MODEL_PATH = os.path.join(os.environ['HOME'], "models")
os.listdir(MODEL_PATH)

['mask_rcnn_resnet101_atrous_coco_2018_01_28.tar.gz',
 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz',
 'ssd_mobilenet_v2_quantized_300x300_coco_2018_09_14.tar.gz',
 'serving',
 'ssd_mobilenet_v2_coco_2018_03_29.tar.gz',
 'faster_rcnn_resnet50_coco_2018_01_28.tar.gz',
 'ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03.tar.gz',
 'rfcn_resnet101_coco_2018_01_28',
 'ssd_mobilenet_v2_quantized_300x300_coco_2018_09_14',
 'ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03',
 'mask_rcnn_resnet50_atrous_coco_2018_01_28',
 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco_2018_01_28',
 'mask_rcnn_inception_v2_coco_2018_01_28.tar.gz',
 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28',
 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03',
 'mask_rcnn_inception_v2_coco_2018_01_28',
 'faster_rcnn_resnet101_coco_2018_01_28.tar.gz',
 'ssd_mobilenet_v1_coco_2018_01_28.tar.gz',
 'ssd_mobilenet_v1_0.

In [4]:
CV_MODEL = 'faster_rcnn_resnet101_lowproposals_coco_2018_01_28'
saver = tf.train.import_meta_graph(os.path.join(MODEL_PATH, CV_MODEL, 'model.ckpt.meta'))
saver.restore(sess, tf.train.latest_checkpoint(os.path.join(MODEL_PATH, CV_MODEL)))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/reddi-rtx/models/faster_rcnn_resnet101_lowproposals_coco_2018_01_28/model.ckpt


In [5]:
input_nodes = [v for v in tf.get_default_graph().as_graph_def().node
               if "Placeholder" in v.op]
output_nodes = [v for v in tf.get_default_graph().as_graph_def().node
                if "detection" in v.name]

In [6]:
graph = tf.get_default_graph()

In [7]:
input_tensor = graph.get_tensor_by_name('image_tensor:0')
output_tensors = dict(
    bboxes = graph.get_tensor_by_name('detection_boxes:0'),
    classes = graph.get_tensor_by_name('detection_classes:0'),
    n = graph.get_tensor_by_name('num_detections:0'),
    scores = graph.get_tensor_by_name('detection_scores:0'),
)

### Load test video

Videos are usually in the `~/videos` directory.

In [8]:
import cv2
VIDEO_PATH = os.path.join(os.environ["HOME"], "videos")
os.listdir(VIDEO_PATH)

['nacho.mkv',
 'scicenplaza_test.mkv',
 'cs121lecture1.mp4',
 'boazobfuscation.mp4',
 '4-13-19-3-48pm.mkv',
 'testnight.mkv',
 'testday.mkv',
 'chicken']

## Processing Entire Video

Now we process entire video and add the bboxes, scores, classes to a list.

In [9]:
def pred_from_frame(frames):
    """Takes a list of frames and runs it through our prediction"""
    frame = np.stack(frames)
    output = sess.run(output_tensors, 
         feed_dict={input_tensor: frame})
    bboxes, score, classes = output['bboxes'], output['scores'], output['classes']
    return bboxes, score, classes

In [10]:
import time
import tqdm
def process_video(video_path, batch_size=32, num_batches=100):
    cap = cv2.VideoCapture(video_path)
    all_scores, all_classes, all_bboxes = [], [], []
    start_time = time.time()
    video_running = True
    for _ in tqdm.tqdm(range(num_batches)):
        frames = []
        for _ in range(batch_size):
            ret, frame = cap.read()
            if not ret:
                print("Video finished")
                video_running = False
                break
            frames.append(frame)
        bboxes, scores, classes = pred_from_frame(frames)
        all_scores.append(scores)
        all_bboxes.append(bboxes)
        all_classes.append(classes)

        if not video_running:
            break
            
    print("Total time: {} seconds".format(int(time.time() - start_time)))
    full_bboxes = np.row_stack(all_bboxes)
    full_scores = np.row_stack(all_scores)
    full_classes = np.row_stack(all_classes)
    return full_bboxes, full_scores, full_classes

In [11]:
VIDEO_NIGHT = os.path.join(VIDEO_PATH, 'testnight.mkv')
VIDEO_DAY = os.path.join(VIDEO_PATH, 'testday.mkv')

In [12]:
import numpy as np
from pathlib import Path

def make_predictions(videoname):
    video = os.path.join(VIDEO_PATH, videoname)
    BATCH_SIZE = 32
    NUM_BATCHES = 250
    start_time = time.time()
    bboxes, scores, classes = process_video(
        video, batch_size=BATCH_SIZE, num_batches=NUM_BATCHES)
    end_time = time.time()
    elapsed_time = np.array([end_time - start_time]) / BATCH_SIZE / NUM_BATCHES
    
    # saving everything
    filename = '{}-{}'.format(os.path.splitext(videoname)[0], CV_MODEL)
    np.savez(filename, bboxes=bboxes, scores=scores, classes=classes,
             inftime=elapsed_time)

In [13]:
make_predictions("testday.mkv")
make_predictions("testnight.mkv")

  0%|          | 0/250 [00:00<?, ?it/s]

Total time: 366 seconds


100%|██████████| 250/250 [06:02<00:00,  1.45s/it]

Total time: 362 seconds
